<a href="https://colab.research.google.com/github/Aravind-11/AI-Gaming/blob/main/Copy_of_multi_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import numpy as np
import gym

class Football:  # The class encapsulating the environment
    '''
    Actions [0 : Stand, 1 : Up, 2 : Right, 3 : Down, 4 : Left]
    These are the representing no.s for the mentioned actions
    '''

    def __init__(self, length=8, width=8, goalPositions=[4, 4]):
        
        # The player start at random locations
        
        self.pA=np.array([np.random.randint(length), np.random.randint(length)]) 
        self.pB=np.array([np.random.randint(length), np.random.randint(length)]) 
        
        self.h = length   # Length of the Football Pitch    
        self.w = width    # Width of the Football Pitch
        
        self.goalPositions = np.array(goalPositions)   # This means that the middle 4 positions at the right and left are the goals
        
        self.reward = np.array([0,0])
        
                                  # Initially the reward is 0
        
        self.observation=np.random.rand(5,)
        self.done = bool(0)    
        self.observation_space=gym.spaces.Box(low=-8, high=8,
                                        shape=(5,), dtype=np.float32)
        self.ballOwner = np.random.randint(2)
        self.action_space=gym.spaces.Discrete(5)
    

    def isInBoard(self, x, y):
        if(x<0 or x>8):
          return 0
        if(y<0 or y>8):
          return 0 
        return 1
    
    def actionToMove(self, action):
        switcher = {
            0: [0, 0],
            1: [0, 1],
            2: [1, 0],
            3: [0, -1],
            4: [-1, 0],
        }
        return switcher.get(action)

In [50]:
class Agent_A(Football,gym.Env):
  def __init__(self, length=8, width=8, goalPositions=[2, 6]):
    super().__init__()
    
    
  def reset(self):
        self.done = bool(0)
        
        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pB = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        return np.array((*self.pA,*self.pB,(4-self.pA[0]),(4-self.pA[1]))).astype(np.float32)
  def step(self, action):
        if self.done == 1:
          self.reset()
        self.move(action)                   # We chose the first player at random !!! important thing to consider - how to choose first player . 
        if self.done == 1:
          return self.observation, self.reward[0], self.done

        return self.observation, self.reward[0].astype(float), self.done,{}
  
  def move(self, action):
        
        newPosition = self.pA + self.actionToMove(action)

        # If it's opponent position
        if (newPosition == self.pB).any():
            self.ballOwner = 1
            self.reward[0]=-20
            self.reward[1]=20
        # If it's a goal
        elif self.ballOwner is 0 and self.isInGoal(*newPosition) >= 0:
            self.done = 1
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward[0] =  -0.1 * ((((abs(newPosition[0]-7)+0.1)+(abs(newPosition[1]-4)+0.1))) -  (self.ballOwner) )
            self.pA = newPosition
        self.observation=np.array((*self.pA,*self.pB,self.ballOwner)).astype(np.float32)
        return -1
  def reset(self):
        self.done = bool(0)
        self.reward = np.array([0,0])
        
        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pB = np.array([np.random.randint(self.h), np.random.randint(self.h)]) 
        return np.array((*self.pA,*self.pB,self.ballOwner)).astype(np.float32)
  def render(self):
        board = ''
        for y in range(self.h)[::-1]:
            for x in range(self.w):
                if ([x, y] == self.pA).all():
                    board += 'A' if ballOwner is 0 else 'a'
                elif ([x, y] == self.pB).all():
                    board += 'B' if ballOwner is 1 else 'b'
                else:
                    board += '-'
            board += '\n'

  def isInGoal(self, x, y):
        g1, g2 = self.goalPositions
        if (g1 <= y <= g2):
            if x == 0:
                self.done = bool(1)
                self.reward[0] = -20 # if the ball reaches the right goal post, then the rewards shall be -1
                return 1 
            elif x == (self.w-1):
                self.done = bool(1)
                self.reward[0] = 20 # if the ball reaches the right goal post, then the rewards shall be 1
                return 0
        return -1

  def seed():
      return 0 
  def metadata(x):
      return 0 
  def legal_actions(self):
    return gym.spaces.Discrete(5)
  def close(self):
    pass

In [59]:
env=Agent_B(Football,gym.Env)

In [60]:
check_env(env, warn=True)

In [39]:
env.step(2)

(array([2., 6., 1., 4., 1.], dtype=float32), 0, False)

In [58]:
class Agent_B(Football,gym.Env):
  def __init__(self, length=8, width=8, goalPositions=[2, 6]):
    super().__init__()
    
    
  def reset(self):
        self.done = bool(0)
        
        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pB = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        return np.array((*self.pB,*self.pA,self.ballOwner)).astype(np.float32)
  def step(self, action):
        if self.done == 1:
          self.reset()
        self.move(action)                   # We chose the first player at random !!! important thing to consider - how to choose first player . 
        if self.done == 1:
          return self.observation, self.reward[1], self.done

        return self.observation, self.reward[1].astype(np.float), self.done,{}
  
  def move(self, action):
        
        newPosition = self.pB + self.actionToMove(action)

        # If it's opponent position
        if (newPosition == self.pA).any():
            self.ballOwner = 0
            self.reward[1]=-20
            self.reward[0]=20
        # If it's a goal
        elif self.ballOwner is 1 and self.isInGoal(*newPosition) >= 0:
            self.done = 1
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward[1] =  -0.1 * ((((abs(newPosition[0]-7)+0.1)+(abs(newPosition[1]-4)+0.1))) + (self.ballOwner) )
            self.pB = newPosition
        self.observation=np.array((*self.pB,*self.pA,self.ballOwner)).astype(np.float32)
        return -1
  def reset(self):
        self.done = bool(0)
        self.reward = np.array([0,0])

        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pB = np.array([np.random.randint(self.h), np.random.randint(self.h)]) 
        return np.array((*self.pB,*self.pA,self.ballOwner)).astype(np.float32)
  def render(self):
        board = ''
        for y in range(self.h)[::-1]:
            for x in range(self.w):
                if ([x, y] == self.pA).all():
                    board += 'A' if ballOwner is 0 else 'a'
                elif ([x, y] == self.pB).all():
                    board += 'B' if ballOwner is 1 else 'b'
                else:
                    board += '-'
            board += '\n'
  def isInGoal(self, x, y):
        g1, g2 = self.goalPositions
        if (g1 <= y <= g2):
            if x == 0:
                self.done = bool(1)
                self.reward[1] = -20 # if the ball reaches the right goal post, then the rewards shall be -1
                return 1 
            elif x == (self.w-1):
                self.done = bool(1)
                self.reward[1] = 20 # if the ball reaches the right goal post, then the rewards shall be 1
                return 0
        return -1
  def seed():
      return 0 
  def metadata(x):
      return 0 
  def legal_actions(self):
    return gym.spaces.Discrete(5)
  def close(self):
    pass

In [29]:
env=Agent_B(Football,gym.Env)

In [ ]:
from stable_baselines import DQN, PPO2, A2C, ACKTR
from stable_baselines.common.cmd_util import make_vec_env

# Instantiate the env
#env = GoLeftEnv(grid_size=10)
# wrap it
env = make_vec_env(lambda: env, n_envs=1)

In [ ]:
# Train the agent
model = PPO2('MlpPolicy', env, verbose=1).learn(5000)

--------------------------------------
| approxkl           | 0.0002893372  |
| clipfrac           | 0.0           |
| ep_len_mean        | 22            |
| ep_reward_mean     | -6.87         |
| explained_variance | -0.00201      |
| fps                | 317           |
| n_updates          | 1             |
| policy_entropy     | 1.6090206     |
| policy_loss        | -0.0051693367 |
| serial_timesteps   | 128           |
| time_elapsed       | 3.5e-05       |
| total_timesteps    | 128           |
| value_loss         | 42.95536      |
--------------------------------------
--------------------------------------
| approxkl           | 0.00027678287 |
| clipfrac           | 0.0           |
| ep_len_mean        | 48            |
| ep_reward_mean     | -24           |
| explained_variance | -0.0126       |
| fps                | 895           |
| n_updates          | 2             |
| policy_entropy     | 1.6070211     |
| policy_loss        | -0.002494718  |
| serial_timesteps   | 25

In [ ]:
obs = env.reset()
n_steps = 20
for step in range(n_steps):
  action, _ = model.predict(obs, deterministic=True)
  print("Step {}".format(step + 1))
  print("Action: ", action)
  obs, reward, done, info = env.step(action)
  print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render(mode='console')
  if done:
    # Note that the VecEnv resets automatically
    # when a done signal is encountered
    print("Goal reached!", "reward=", reward)
    break

Step 1
Action:  [1]
obs= [[3. 2. 1. 2.]] reward= [-0.32] done= [False]
--------
--------
--------
--------
--------
---A----
--------
--------

Step 2
Action:  [1]
obs= [[3. 3. 1. 1.]] reward= [-0.22] done= [False]
--------
--------
--------
--------
---A----
--------
--------
--------

Step 3
Action:  [2]
obs= [[4. 3. 0. 1.]] reward= [-0.12] done= [False]
--------
--------
--------
--------
----A---
--------
--------
--------

Step 4
Action:  [2]
obs= [[ 5.  3. -1.  1.]] reward= [-0.22] done= [False]
--------
--------
--------
--------
-----A--
--------
--------
--------

Step 5
Action:  [2]
obs= [[ 6.  3. -2.  1.]] reward= [-0.32] done= [False]
--------
--------
--------
--------
------A-
--------
--------
--------

Step 6
Action:  [1]
obs= [[ 6.  4. -2.  0.]] reward= [-0.22] done= [False]
--------
--------
--------
------A-
--------
--------
--------
--------

Step 7
Action:  [2]
obs= [[ 6.  3. -2.  1.]] reward= [20.] done= [ True]
--------
--------
--------
--------
------A-
------

In [ ]:


def move(self, action):
        
        newPosition = self.positions[player] + self.actionToMove(action)

        # If it's opponent position
        if (newPosition == self.pA).all():
            self.ballOwner = 1
            self.reward=-20
        # If it's a goal
        elif self.ballOwner is 0 and self.isInGoal(*newPosition) >= 0:
            self.done = 1
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward =  -0.1 * ((((abs(newPosition[0]-self.goalPositions[0])+0.1)+(abs(newPosition[1]-self.goalPositions[1])+0.1))) -  (self.ballOwner) )
            self.pA = newPosition
        return -1












def step(self, actionA, actionB):
        if self.done == 1:
          self.reset()
          
        #first = self.choosePlayer()
        #actions = [actionA, actionB]
        self.move(action)                   # We chose the first player at random
        if self.done == 1:
          return self.observation, self.reward, self.done
        #self.move(1 - first, actions[1 - first])

        return self.observation, self.reward, self.done




    
    
  def step(self, action):
        #print('action',action)
        if self.done == bool(1):
          self.reset()
        
        self.move(action)                   # We chose the first player at random
        if self.done == bool(1):
          return self.observation, self.reward, self.done,{}
        if not self.done:
            self.current_player_num = 0
        return self.observation,self.reward, self.done,{}
  
  def move(self, action):
        
        newPosition = self.pA + self.actionToMove(action)
        
       
        if self.isInGoal(*newPosition) >= 0:
            self.done = bool(1)
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward = -0.1 * ((((abs(newPosition[0]-self.goalPositions[0])+0.1)+(abs(newPosition[1]-self.goalPositions[1])+0.1))))
            self.pA = newPosition
        
        self.observation=np.array((*self.pA,(4-self.pA[0]),(4-self.pA[1]))).astype(np.float32)
        return -1
    def reset(self):
        self.done = bool(0)
        self.reward = 0
        
        self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        self.pB = np.array([np.random.randint(self.h), np.random.randint(self.h)])
        return np.array((*self.pA,*self.pB,(4-self.pA[0]),(4-self.pA[1]))).astype(np.float32)

    def step(self, action):
        if self.done == bool(1):
          self.reset()
        self.move(first, action)                   # We chose the first player at random
        if self.done == bool(1):
          return self.observation, self.reward, self.done
        if not done:
            self.current_player_num = (self.current_player_num + 1) % 2   
        return self.observation,self.reward, self.done

    def move(self, player, action):
        opponent = 1 - player
        
        newPosition = self.pA + self.actionToMove(action)
        
        if self.ballOwner is player and self.isInGoal(*newPosition) >= 0:
            self.done = bool(1)
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.positions[player] = newPosition
        if(self.ballOwner!=0):
          self.reward=-1
        return -1

    def actionToMove(self, action):
        switcher = {
            0: [0, 0],
            1: [0, 1],
            2: [1, 0],
            3: [0, -1],
            4: [-1, 0],
        }
        return switcher.get(action)

    def isInGoal(self, x, y):
        g1, g2 = self.goalPositions
        if (g1 <= y <= g2):
            if x == 0:
                self.done = bool(1)
                self.reward = -20 # if the ball reaches the right goal post, then the rewards shall be -1
                return 1 
            elif x == (self.w-1):
                self.done = bool(1)
                self.reward = 20 # if the ball reaches the right goal post, then the rewards shall be 1
                return 0
        return -1

    def isInBoard(self, x, y):
        if(x<0 or x>8):
          return 0
        if(y<0 or y>8):
          return 0 
        return 1
        

    #def choosePlayer(self):
    #    return np.random.randint(0, 2)
    def render(self,mode="human"):
        

        board = ''
        for y in range(self.h)[::-1]:
            for x in range(self.w):
                if ([x, y] == self.pA).all():
                    board += 'A' 
                
                else:
                    board += '-'
            board += '\n'

        print(board)

class modf_football(Football,gym.Env):
  def __init__(self, length=8, width=8, goalPositions=[2, 6]):
    super().__init__()
    self.observation_space=gym.spaces.Box(low=-8, high=8,
                                        shape=(4,), dtype=np.float32)
    self.reward=0
    self.action_space=gym.spaces.Discrete(5)
    self.name='Football'
    self.current_player_num=0
    self.observation=np.random.rand(4,)
    self.pA = np.array([np.random.randint(self.h), np.random.randint(self.h)])
  #modifying the step and move function to get the updated reward system
  def step(self, action):
        #print('action',action)
        if self.done == bool(1):
          self.reset()
        
        self.move(action)                   # We chose the first player at random
        if self.done == bool(1):
          return self.observation, self.reward, self.done,{}
        if not self.done:
            self.current_player_num = 0
        return self.observation,self.reward, self.done,{}
  
  def move(self, action):
        
        newPosition = self.pA + self.actionToMove(action)
        
       
        if self.isInGoal(*newPosition) >= 0:
            self.done = bool(1)
            return 1 - self.isInGoal(*newPosition)
        # If it's in the board
        elif self.isInBoard(*newPosition):
            self.reward = -0.1 * ((((abs(newPosition[0]-self.goalPositions[0])+0.1)+(abs(newPosition[1]-self.goalPositions[1])+0.1))))
            self.pA = newPosition
        
        self.observation=np.array((*self.pA,(4-self.pA[0]),(4-self.pA[1]))).astype(np.float32)
        return -1
  def seed():
      return 0 
  def metadata(x):
      return 0 
  def legal_actions(self):
    return gym.spaces.Discrete(5)
  def close(self):
    pass

In [11]:
# Stable Baselines only supports tensorflow 1.x for now
%tensorflow_version 1.x
!pip install stable-baselines[mpi]==2.10.0

TensorFlow 1.x selected.
     |████████████████████████████████| 248 kB 5.2 MB/s 
  Attempting uninstall: stable-baselines
    Found existing installation: stable-baselines 2.2.1
    Uninstalling stable-baselines-2.2.1:
      Successfully uninstalled stable-baselines-2.2.1


In [19]:
env.step(2)

TypeError: ignored

In [ ]:
print("Observation space:", env.observation_space)
print("Shape:", env.observation_space.shape)
# Discrete(2) means that there is two discrete actions
print("Action space:", env.action_space)

# The reset method is called at the beginning of an episode
obs = env.reset()
# Sample a random action
action = env.action_space.sample()
print("Sampled action:", action)
obs, reward, done, info = env.step(action)
# Note the obs is a numpy array
# info is an empty dict for now but can contain any debugging info
# reward is a scalar
print(obs, reward, done, info)

Observation space: Box(-8.0, 8.0, (4,), float32)
Shape: (4,)
Action space: Discrete(5)
Sampled action: 2
[ 8.  5. -4. -1.] 0.014285714285714285 False {}


In [34]:
from stable_baselines.common.env_checker import check_env

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [ ]:
4/2

2.0